In [16]:
# werkzeug
# shapely=1.7.1
# geopandas
# pandas
# contextily
# matplotlib
# docxtpl
# docx

In [1]:
import sys
sys.path.append('../_amigocloud')

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
from werkzeug.wrappers import Request, Response
from jinja2 import Template
from datetime import datetime, timedelta
import collections
from amigocloud import AmigoCloud
from datetime import datetime
from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests
import qrcode
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)

In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../unidad_local.txt')
ruta = ruta.readlines()[0]

In [5]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [6]:
def crear_cite(cod_canhero, nom_canhero, tipo_reporte):
    ruta_api_google = ruta + '\_keys\client_secret_google_sheets.json'
    # Configura las credenciales
    scope = ["https://spreadsheets.google.com/feeds",
             'https://www.googleapis.com/auth/spreadsheets', 
             "https://www.googleapis.com/auth/drive.file", 
             "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(ruta_api_google, scope)
    client = gspread.authorize(creds)
    # Abre la hoja de cálculo con ID y la hoja específica
    spreadsheet = client.open_by_key("14QiX-Kkz5OY08mYsVxrGXk0offCuDVdBzX07vT4f8mQ")
    sheet = spreadsheet.sheet1
    # Lee los datos
    data_sheet = sheet.get_all_records()
    df = pd.DataFrame(data_sheet)
    # Encuentra el número más grande y lo incrementamos y agrega un nuevo registro
    max_value = df['No'].max()
    no = int(max_value + 1)
    # llenamos los demas campos
    fecha = datetime.now().strftime("%m/%d/%Y")
    cod_ca = int(cod_canhero)
    nom_ca = nom_canhero
    tipo = tipo_reporte
    #agregar el registro
    sheet.append_row([no, fecha, cod_ca, nom_ca, tipo])
    return no

In [7]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_307560 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = (datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d') + timedelta(days=1)).strftime('%d/%m/%Y')
    credito_data['fecha_1ra_insp'] = datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    
    cite = crear_cite(credito_data['canhero'].split(' / ')[0], credito_data['canhero'].split(' / ')[1], 'CRSI')
    credito_data['cite_1ra_insp'] = cite
    
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    query_propiedades = {'query': 'select * from dataset_307562 where insp_ref_prop = \'{insp_ref_prop}\''.format(insp_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_propiedades)
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': 'select * from dataset_307563 where prop_ref_lote=\'{prop_ref_lote}\''.format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_46845 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))
    
    
    # GENERAR QR
    texto = '''
--UTEA--
--1RA INSPECCION CREDITO SIEMBRA--

Codigo de Inspeccion: {cite}
Fecha de Inspeccion: {fecha}
Nombre de Canero: {canero}
Codigo Canero: {cod_ca}
Total Solicitado: {soli}
Total Area Insp.: {total_area} ha
Estado de Insp.: {estado}
Tecnico Responsable: {tecnico}'''\
    .format(
        cite=str(credito.cite_1ra_insp) + '-' + credito.fecha_1ra_insp.split('/')[2], 
        fecha=credito.fecha_1ra_insp, 
        canero=credito.canhero.split(' / ')[1],
        cod_ca=credito.canhero.split(' / ')[0],
        soli=credito.solicitado,
        total_area=credito.total_1ra_insp,
        estado=credito.estado_1ra_insp,
        tecnico=credito.respon_1ra_insp)
    # Generar el código QR
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(texto)
    qr.make(fit=True)
    
    qr_img = qr.make_image(fill_color="black", back_color="white")
    
    cod_ca = credito.canhero.split(' / ')[0]
    nom_ca = credito.canhero.split(' / ')[1]
    filename_qr = "qr_" + cod_ca + '_CRS_' + credito.fecha_informe.replace('/', '-') + '_' + nom_ca
    ruta_qr = ruta + '/planos/' + filename_qr + '.png'
    # Guardar el código QR en un archivo
    qr_img.save(ruta_qr)

    
    
    
    

        
    # GENERAR REPORTE
    doc = DocxTemplate(ruta + "/templates/tpl_inf_credito_1ra.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open(ruta + '/fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta+'/fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_1ra_insp == 'Rogelio Acuña Rodríguez':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_rogelio.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_jaldin.png', width=Mm(60))
    # se crea obj inline par imagen QR
    qr_file = docxtpl.InlineImage(doc, image_descriptor=ruta_qr, width=Mm(30))
    
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon, 'qr':qr_file}
    doc.render(context)
    cod_ca = credito.canhero.split(' / ')[0]
    nom_ca = credito.canhero.split(' / ')[1]
    print(credito)
    file_name = "_" + cod_ca + '_CRS_' + credito.fecha_informe.replace('/', '-') + '_' + nom_ca
    doc.save(ruta + '/informes/' + file_name + '.docx')

In [8]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    path = ruta+'/planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(path, dpi = 300, bbox_inches='tight')
    plt.clf()
    return path

In [10]:
semana = 'S05'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_307560 where semana_1ra = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32672/sql'
select = amigocloud.get(project, query)
data = select['data']

In [11]:
data

[{'cod': '18088', 'nom': 'FRERKING ORTIZ RICARDO'},
 {'cod': '41594', 'nom': 'AGROPECUARIA CAMPO DULCE S.R.L.'},
 {'cod': '18028', 'nom': 'RIVERO GUTIERREZ FRANCISCO'},
 {'cod': '4834', 'nom': 'GUTIERREZ MARTHA S. AGUILERA DE'},
 {'cod': '20055', 'nom': 'FRANCO ARIAS EDGAR'},
 {'cod': '40256', 'nom': 'INSERVA DAMIANA S.R.L.'}]

In [12]:
#lista = [1068, 18062, 10739, 41955, 2392, 15162, 2517, 8706, 15085, 430, 874, 5498, 2505, 2274]
lista = [int(i['cod']) for i in data]

In [13]:
print(len(lista), 'informes')

6 informes


In [14]:
lista

[18088, 41594, 18028, 4834, 20055, 40256]

In [16]:
lista = [41594]

In [17]:
for i in lista:
    generar_reporte(i)
    print('======================', i)

L39 ba83e4f7c5614e389fac272634583391
L28 1c7c60604fa640969bd09c8ea6d8cbe9
L30 4b158480e782490daaf1239cb76b88d2
L32 4de9c77bb2914c15869d88894ce2e570
L34 12f101d4f0d44a0d8e9180c6b9ff84a2
L1 5bac776af1d24d6a88588a830a8998d2
L12B 99d6a14ec9af4996970a1aefba20809d
L23 f12c00a953f640d08b06a05ce389aa77
L17A bea729b775e14fc19dad40726f9cb234
L18 ed984b3a12bd4c62b09fdba682265408
L43 d35f204d6ad045d09faa2abe90b03335
L26 67dc369b44494c3aba1f5ec8f6a4043c
L45 6b0f522a83f04108a8ac83daed1731b6
L33 084cda3e68a448c1b6f1591a6ab7af87
L35 6ebc0726ad19409e832391203967613f
L36 dde12730cc154e019d371f4be7dc9006
L38 663796076d07463dbcef008e8dce8cf3
L41 5afb96a794bc41d0aaae1afde46789a4
L5 4056f4c7648e4c148959494db799f235
L46 3a274a5e014241dabd20e7f89db1e473
L6 cfe16f5507f8480c84f973cdcaab1362
L7 b82c09b44cc24837bf0cac26f0c23fe8
L20 411f2d2818e345bb9c7091d9c4b54433
L27 e45491e9f856406496a367dad3bd582d
L17B b970cae88bd245de9252f9a62277d463
L16B 22e1043ad8634954aa2b6124c21bd6f0
L29 bca03f4159294bee9d5b161bf065213c
L

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>